In [1]:
import re
import numpy as np

from itertools import chain

In [2]:
import pandas as pd

### Задание 1 (10 баллов)

Прочитайте данные из emails.csv. В колонке message содержится информация о письме. Вам нужно сформировать несколько таблиц по данным для комфортной работы.


In [3]:
emails = pd.read_csv('emails2.csv')

Открыв данные, воспользуемся библиотекой email, чтобы распарсить письма: кажется, наши данные -- это письма в том виде, в каком ои путешествуют по сети, соответственно, удобнее воспользоваться специальными инструментами для их обработки, вместо того, чтобы писать огромные регулярки.

In [4]:
import email
from email.utils import parseaddr


In [5]:
ms = [email.message_from_string(m)["To"] for m in emails["message"][0:100]]

In [6]:
emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
file       517401 non-null object
message    517401 non-null object
dtypes: object(2)
memory usage: 7.9+ MB


In [7]:
ids = {}
table = []
I = 0
for m in emails["message"]:
    obj = email.message_from_string(m)
    to = obj["To"]
    fro = obj["From"]
    try:
        if to != None:
            es = re.split('[ \r\n\t]*,[ \r\n\t]*', to)
            eg = re.split('[ \r\n\t]*,[ \r\n\t]*', fro)
            ns = re.split(',', obj["X-To"])
            ng = re.split(',', obj["X-From"])
            for i in range(len(es)):
                e = es[i]
                if not e in ids:
                    I+=1
                    ids[e] = I
                    table.append( (I, e, ns[i]) )
            for i in range(len(eg)):
                e = eg[i]
                if not e in ids:
                    I+=1
                    ids[e] = I
                    table.append( (I, e, ng[i]) )
    except TypeError:
        continue

Получили первую таблицу: почта, имя и айди для каждого участника коммуникации.

In [8]:
table [:10]

[(1, 'tim.belden@enron.com', 'Tim Belden <Tim Belden/Enron@EnronXGate>'),
 (2, 'phillip.allen@enron.com', 'Phillip K Allen'),
 (3,
  'john.lavorato@enron.com',
  'John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>'),
 (4, 'leah.arsdall@enron.com', 'Leah Van Arsdall'),
 (5, 'randall.gay@enron.com', 'Randall L Gay'),
 (6, 'greg.piper@enron.com', 'Greg Piper'),
 (7, 'david.l.johnson@enron.com', 'david.l.johnson@enron.com'),
 (8, 'john.shafer@enron.com', ' John Shafer'),
 (9, 'joyce.teixeira@enron.com', 'Joyce Teixeira'),
 (10, 'mark.scott@enron.com', 'Mark Scott')]

In [9]:
names_df = pd.DataFrame(table)

In [10]:
names_df.columns = ['id', 'e-mail', 'name']

In [11]:
names_df[:10]

,id,e-mail,name
0,1,tim.belden@enron.com,Tim Belden <Tim Belden/Enron@EnronXGate>
1,2,phillip.allen@enron.com,Phillip K Allen
2,3,john.lavorato@enron.com,John J Lavorato <John J Lavorato/ENRON@enronXg...
3,4,leah.arsdall@enron.com,Leah Van Arsdall
4,5,randall.gay@enron.com,Randall L Gay
5,6,greg.piper@enron.com,Greg Piper
6,7,david.l.johnson@enron.com,david.l.johnson@enron.com
7,8,john.shafer@enron.com,John Shafer
8,9,joyce.teixeira@enron.com,Joyce Teixeira
9,10,mark.scott@enron.com,Mark Scott


Теперь делаем таблицу с письмами.

In [12]:
#ids = {}
letters = []
I = 0
for m in emails["message"]:
    obj = email.message_from_string(m)
    Date = obj["Date"]
    id_get = obj["To"]
    id_sent = obj["From"]
    payload = ""
    try:
        if obj.is_multipart: 
            payload = "".join([pl for pl in obj.get_payload()])
        else:
            payload = obj.get_payload();
        to_ids = [ids[e] for e in re.split('[ \r\n\t]*,[ \r\n\t]*' , obj["To"])]
        letters.append((payload, Date, ids[obj["From"]], to_ids))
    except (TypeError, KeyError):
        continue

In [43]:
letters[0]

('Here is our forecast\n\n ', 'Mon, 14 May 2001 16:39:00 -0700 (PDT)', 2, [1])

In [13]:
letters_df = pd.DataFrame(letters)

In [14]:
letters_df.columns = ['text', 'Date','from-id', 'to-id']

In [15]:
letters_df[:20]

,text,Date,from-id,to-id
0,Here is our forecast\n\n,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",2,[1]
1,Traveling to have a business meeting takes the...,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",2,[3]
2,test successful. way to go!!!,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",2,[4]
3,"Randy,\n\n Can you send me a schedule of the s...","Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",2,[5]
4,Let's shoot for Tuesday at 11:45.,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",2,[6]
5,"Greg,\n\n How about either next Tuesday or Thu...","Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",2,[6]
6,Please cc the following distribution list with...,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",2,"[7, 8]"
7,any morning between 10 and 11:30,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",2,[9]
8,1. login: pallen pw: ke9davis\n\n I don't thi...,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",2,[10]
9,---------------------- Forwarded by Phillip K ...,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",2,[11]


In [16]:
letters_info = letters_df.groupby('from-id')['text'].nunique()

In [27]:
#lshape = letters_info.shape

In [41]:
lshape = (len(ids), len(ids))
lshape
table[0][1]

'tim.belden@enron.com'

Задание 2 (15 баллов)

Сформируйте матрицу $A = (a_{ij})_{i,j = 1}^N$, где $a_{ij}$ - количество писем, которое индивид $i$ отправил индивиду $j$, $N$ - количество индивидуумов

Далее из матрицы $A$ получим симметричную матрицу, которая будет показывать нам частоту коммуникаций между каждой парой индивидов: $A+A^T$. Постройте по данной матрице не направленный граф, где узлы должны называться по именам, которые вы извлекли из X-From и X-To

In [18]:
from itertools import chain

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt

from itertools import combinations
from sklearn.preprocessing import LabelEncoder


In [47]:
from scipy.sparse import csr_matrix

In [42]:
table[0]

(1, 'tim.belden@enron.com', 'Tim Belden <Tim Belden/Enron@EnronXGate>')

In [48]:
def mat(letters, shape):
    pre = {}
    row = np.array([])
    col = np.array([])
    data = np.array([])
    for letter in letters:
        fr = letter[2]
        for to in letter[3]:
            s = "{},{}".format(fr, to)
            if s in pre:
                pre[s]+=1
            else:
                pre[s]=1
    for s in pre:
        d = s.split(',')
        fr = int(d[0])-1
        to = int(d[1])-1
        row = np.append(row, fr)
        col = np.append(col, to)
        data = np.append(data, pre[s])
    print(row.size, col.size, data.size)
    print(shape)
    return csr_matrix((data, (row, col)), shape=shape)


In [49]:
lmatrix = mat(letters, lshape)

311206 311206 311206
(79736, 79736)


In [50]:
np.save('letter_matrix', lmatrix)

In [ ]:
plt.matshow(lmatrix)
plt.colorbar()

plt.xticks(np.arange(shape), le.classes_, rotation=90)
plt.yticks(np.arange(lshape), le.classes_)

plt.show()

Задание 3 (25 баллов)

Оцените важность отдельных узлов, используя следующие меры:

Degree centrality
Closeness centrality
Betweenness centrality
Eigenvector centrality
Подробнее читайте в статье: УЗЛЫ В СОЦИАЛЬНЫХ СЕТЯХ: МЕРЫ ЦЕНТРАЛЬНОСТИ И РОЛЬ В СЕТЕВЫХ ПРОЦЕССАХ

Для каждой меры центральности выведите топ пользователей

In [ ]:
import networkx as nx

G = nx.Graph(A/A.max())

plt.figure(figsize=(15, 15))
nx.draw(G, labels = dict(zip(range(N), le.classes_)), with_labels = True)
plt.show()

Бонусное задание (25 баллов)

Выделите темы, которые обсуждаются в письмах.

Выберите одну из мер центральности и для топа пользователей по данной мере укажите, какие темы они обсуждают чаще всего и с кем.